# LeNeT-5

### LeNeT-5 based architecture for deep learning of MNIST dataset

Import libraries

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

Define hyperparameters

In [2]:
num_classes = 10 # number of output classes discrete range [0,9]
num_epochs = 2 # number of times which the entire dataset is passed throughout the model
batch_size = 100 # the size of input data took for one iteration
lr = 1e-3 # size of step 

Download and load MNIST data

Resize, transform and normalize

In [3]:

train_data = dsets.MNIST(root = './data',
                            train = True,
                            transform = transforms.Compose([
                                  transforms.Resize((32,32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                            download = True)


test_data = dsets.MNIST(root = './data',
                                            train = False,
                                            transform = transforms.Compose([
                                                    transforms.Resize((32,32)),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                            download=True)

train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size, 
                                      shuffle = False)

Download and load MNIST data

In [4]:
from torch.nn import Module
from torch import nn

class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

Build the model

In [5]:
net = Net()
if torch.cuda.is_available():
  net.cuda()

Define loss function and the optimizer

In [6]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

Train the model

In [7]:
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_gen):
    labels = Variable(labels).cuda()
    
    outputs = net(images.cuda().float())
    loss = loss_function(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print('Epoch [%d/%d], Step [%d/%d], Loss: %.3f'
                 %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, loss.data.item()))


Epoch [1/2], Step [100/600], Loss: 0.251
Epoch [1/2], Step [200/600], Loss: 0.303
Epoch [1/2], Step [300/600], Loss: 0.087
Epoch [1/2], Step [400/600], Loss: 0.163
Epoch [1/2], Step [500/600], Loss: 0.064
Epoch [1/2], Step [600/600], Loss: 0.044
Epoch [2/2], Step [100/600], Loss: 0.030
Epoch [2/2], Step [200/600], Loss: 0.012
Epoch [2/2], Step [300/600], Loss: 0.022
Epoch [2/2], Step [400/600], Loss: 0.089
Epoch [2/2], Step [500/600], Loss: 0.070
Epoch [2/2], Step [600/600], Loss: 0.066


Evaluate the accuracy of the model

In [8]:
classes=("0","1","2","3","4","5","6","7","8","9")
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}


correct = 0
total = 0
for images,labels in test_gen:
  
  labels = labels.cuda()
  
  output = net(images.cuda().float())
  _, predicted = torch.max(output,1)

  for label, prediction in zip(labels, predicted):
    if label == prediction:
        correct_pred[classes[label]] += 1
    total_pred[classes[label]] += 1

  correct += (predicted == labels).sum()
  total += labels.size(0)
print('Overall accuracy of the model: %.3f %%' %((100*correct)/(total+1)))

for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.3f} %')

Overall accuracy of the model: 98.590 %
Accuracy for class: 0     is 99.592 %
Accuracy for class: 1     is 99.824 %
Accuracy for class: 2     is 98.159 %
Accuracy for class: 3     is 99.109 %
Accuracy for class: 4     is 99.695 %
Accuracy for class: 5     is 99.215 %
Accuracy for class: 6     is 98.330 %
Accuracy for class: 7     is 98.152 %
Accuracy for class: 8     is 98.768 %
Accuracy for class: 9     is 95.144 %


Evaluate memory consumption and inference time

In [9]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
with profile(activities=[
        ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True, record_shapes=True) as prof:
    with record_function("model_inference"):
      for i ,(images,labels) in enumerate(test_gen):
        outputs = net(images.cuda())

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                        model_inference         1.40%      44.617ms       100.00%        3.187s        3.187s       0.000us         0.00%      54.681ms      54.681ms       6.66 Mb     -32.88 Mb     113.41 Mb    -845.02 M